In [9]:
import pickle
import pandas as pd

In [10]:
df = pd.read_csv('dataset/ratings_Electronics (1).csv',names= ['userId', 'productId', 'Rating', 'timestamp'])

In [11]:
df['Rating'] = df['Rating'].astype(int)
df['userId'] = df['userId'].astype('str')
df['productId'] = df['productId'].astype('str')

df = df.drop(columns=['timestamp'])

In [12]:
filename= 'model/recommendation_model_svd.pkl'
loaded_model= pickle.load(open(filename, 'rb'))

In [16]:
# users ex. A25C2M3QF9G7OQ , A118NVHURNIM28

user_id = "A11MINHURNIM28"

if user_id not in df['userId'].values:
    print('this user is not signed')
else:
    items = df['productId'].unique()

    rated_items = df[df['userId'] == user_id]['productId'].values
    unrated_items = [item for item in items if item not in rated_items]

    predictions = [loaded_model.predict(user_id, item) for item in unrated_items]

    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    # Print the predicted ratings
    for prediction in sorted_predictions[:10]:
        print(f"Item {prediction.iid} - Predicted Rating: {prediction.est}")

this user is not signed


hesitation is defeat